In [2]:
import os
import numpy as np
from scipy import stats
import nibabel as nib

from scipy.spatial.distance import dice
from skimage.metrics import hausdorff_distance
from sklearn.metrics import jaccard_score, cohen_kappa_score
from scipy.stats import wilcoxon, ttest_rel

In [3]:
def load_nifti_file(filepath):
    """ Load a NIfTI file and return the image data as a numpy array. """
    img = nib.load(filepath)
    return img.get_fdata()

def calculate_volume(nifti_file):
    """
    Calculate the volume of non-zero values in a NIfTI file.

    Parameters:
    nifti_file (str): The path to the NIfTI file.

    Returns:
    int: The volume of non-zero values in the NIfTI file.
    """
    img = nib.load(nifti_file)
    data = img.get_fdata()
    volume = np.count_nonzero(data)
    return volume

def calculate_dice(nifti_file1, nifti_file2):
    """
    Calculates the Dice coefficient between two NIfTI files.

    Parameters:
    nifti_file1 (str): Path to the first NIfTI file.
    nifti_file2 (str): Path to the second NIfTI file.

    Returns:
    float: The Dice coefficient between the two NIfTI files.
    """
    img1 = nib.load(nifti_file1)
    data1 = img1.get_fdata()

    img2 = nib.load(nifti_file2)
    data2 = img2.get_fdata()

    intersection = np.sum(data1 * data2)
    dice = (2. * intersection) / (np.sum(data1) + np.sum(data2))
    return dice

def perform_stat_test(path, stat_func):
    """
    Perform a statistical test on the T1CM and T1XFLAIR values.

    Args:
        path (str): The path to the directory containing the subject data.
        stat_func (function): The statistical function to be applied to the T1CM and T1XFLAIR values.

    Returns:
        stat_test (tuple): A tuple containing the test statistic and the p-value.

    """
    t1cm_values = []
    t1xflair_values = []

    # List all directories in the given path
    subject_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

    for id in subject_dirs:
        t1cm_file = os.path.join(path, id, f"{id}_t1cm.nii")
        t1xflair_file = os.path.join(path, id, f"{id}_t1xflair.nii")

        if os.path.exists(t1cm_file) and os.path.exists(t1xflair_file):
            value = stat_func(t1cm_file, t1xflair_file)
            t1cm_values.append(value)

    stat_test = stats.ttest_ind(t1cm_values, t1xflair_values)
    return stat_test

- DSC: Values close to 1 (e.g., >0.8): high similarity
- Jaccard Index: Values close to 1 (e.g., >0.7):  high similarity
- Cohen's Kappa: Values > 0.6: substantial agreement; values > 0.8: almost perfect agreement
- Hausdorff Distance: Lower values: high similarity. values close to 0 are desired.
- Volume: Similar volumes indicate similar segmentation extents. 

In [4]:

def load_nifti_file(filepath):
    """ Load a NIfTI file and return the image data as a numpy array. """
    img = nib.load(filepath)
    return img.get_fdata()

def dice_coefficient(mask1, mask2):
    """ Compute the Dice Similarity Coefficient for two binary masks. """
    mask1 = mask1.astype(bool)
    mask2 = mask2.astype(bool)
    intersection = np.sum(mask1 & mask2)
    size1 = np.sum(mask1)
    size2 = np.sum(mask2)
    if size1 + size2 == 0:
        return 1.0  # Both masks are empty, return perfect similarity.
    return 2.0 * intersection / (size1 + size2)

def jaccard_index(mask1, mask2):
    """ Compute the Jaccard Index for two binary masks. """
    mask1 = mask1.astype(bool).flatten()
    mask2 = mask2.astype(bool).flatten()
    return jaccard_score(mask1, mask2)

def cohen_kappa(mask1, mask2):
    """ Compute Cohen's Kappa for two binary masks. """
    mask1 = mask1.astype(bool).flatten()
    mask2 = mask2.astype(bool).flatten()
    return cohen_kappa_score(mask1, mask2)

def hausdorff_dist(mask1, mask2):
    """ Compute the Hausdorff Distance for two binary masks. """
    return hausdorff_distance(mask1, mask2)

def volume(mask):
    """ Compute the volume (number of voxels) for a binary mask. """
    return int(np.sum(mask))

def process_subjects(subjects_dir, file_ending1 = '_ChP_mask_T1_mdc.nii', file_ending2 = '_ChP_mask_T1xFLAIR.nii'):
    """ Process NIfTI files for all subjects and compute statistical measures for each pair of masks. """
    subjects = [f.name for f in os.scandir(subjects_dir) if f.is_dir()]

    dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1c, volumes_flair = [], [], [], [], [], []

    for subject in subjects:
        t1mc_path = os.path.join(subjects_dir, subject, subject + file_ending1)
        t1xflair_path = os.path.join(subjects_dir, subject, subject + file_ending2)
        
        if not os.path.exists(t1mc_path) or not os.path.exists(t1xflair_path):
            print(f"Skipping subject {subject}: Missing data")
            continue
        
        t1mc_mask = load_nifti_file(t1mc_path)
        t1xflair_mask = load_nifti_file(t1xflair_path)
        
        dsc = dice_coefficient(t1mc_mask, t1xflair_mask)
        jaccard = jaccard_index(t1mc_mask, t1xflair_mask)
        kappa = cohen_kappa(t1mc_mask, t1xflair_mask)
        hausdorff = hausdorff_dist(t1mc_mask, t1xflair_mask)
        vol_t1mc = volume(t1mc_mask)
        vol_t1xflair = volume(t1xflair_mask)
        
        dsc_scores.append(dsc)
        jaccard_scores.append(jaccard)
        kappa_scores.append(kappa)
        hausdorff_scores.append(hausdorff)
        volumes_t1c.append(vol_t1mc)
        volumes_flair.append(vol_t1xflair)
        
        print(f"Subject {subject}: DSC = {dsc:.4f}, Jaccard = {jaccard:.4f}, Kappa = {kappa:.4f}, Hausdorff = {hausdorff:.4f}, Volume T1c = {vol_t1mc}, Volume FLAIR = {vol_t1xflair}")

    return dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1c, volumes_flair

# Define the path to the directory containing the subject data
subjects_dir = '/home/linuxlia/Lia_Masterthesis/data/T1mc_vs_T1xFLAIR'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir)

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs FLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")


Subject 05: DSC = 0.4344, Jaccard = 0.2774, Kappa = 0.4341, Hausdorff = 12.0416, Volume T1c = 4218, Volume FLAIR = 3826
Subject 09: DSC = 0.7661, Jaccard = 0.6208, Kappa = 0.7660, Hausdorff = 9.4340, Volume T1c = 3487, Volume FLAIR = 3267
Subject 12: DSC = 0.5360, Jaccard = 0.3661, Kappa = 0.5359, Hausdorff = 11.3578, Volume T1c = 1666, Volume FLAIR = 1681
Subject 04: DSC = 0.1810, Jaccard = 0.0995, Kappa = 0.1808, Hausdorff = 19.5959, Volume T1c = 2684, Volume FLAIR = 2486
Subject 01: DSC = 0.4423, Jaccard = 0.2839, Kappa = 0.4420, Hausdorff = 10.7703, Volume T1c = 4627, Volume FLAIR = 4440
Subject 06: DSC = 0.5673, Jaccard = 0.3960, Kappa = 0.5672, Hausdorff = 3.3166, Volume T1c = 2874, Volume FLAIR = 2104
Subject 03: DSC = 0.5934, Jaccard = 0.4218, Kappa = 0.5933, Hausdorff = 10.2470, Volume T1c = 2155, Volume FLAIR = 2038
Subject 10: DSC = 0.4532, Jaccard = 0.2930, Kappa = 0.4530, Hausdorff = 14.3178, Volume T1c = 4511, Volume FLAIR = 4518
Subject 11: DSC = 0.3349, Jaccard = 0.2011

In [13]:
# Define the path to the directory containing the subject data
subjects_dir = '/home/linuxlia/Lia_Masterthesis/data/T1mc_vs_T1xFLAIR_controlled'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir)

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs FLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

Subject 05: DSC = 0.9343, Jaccard = 0.8766, Kappa = 0.9342, Hausdorff = 2.2361, Volume T1c = 3752.0, Volume FLAIR = 3824.0
Subject 09: DSC = 0.9621, Jaccard = 0.9269, Kappa = 0.9621, Hausdorff = 3.6056, Volume T1c = 3724.0, Volume FLAIR = 3741.0
Subject 12: DSC = 0.9206, Jaccard = 0.8530, Kappa = 0.9206, Hausdorff = 4.6904, Volume T1c = 1737.0, Volume FLAIR = 1741.0
Subject 04: DSC = 0.8659, Jaccard = 0.7635, Kappa = 0.8659, Hausdorff = 2.2361, Volume T1c = 2577.0, Volume FLAIR = 2486.0
Subject 01: DSC = 0.9682, Jaccard = 0.9384, Kappa = 0.9682, Hausdorff = 2.2361, Volume T1c = 4432.0, Volume FLAIR = 4440.0
Subject 06: DSC = 0.9319, Jaccard = 0.8726, Kappa = 0.9319, Hausdorff = 3.0000, Volume T1c = 2584.0, Volume FLAIR = 2500.0
Subject 03: DSC = 0.9388, Jaccard = 0.8846, Kappa = 0.9388, Hausdorff = 2.8284, Volume T1c = 2127.0, Volume FLAIR = 2038.0
Subject 10: DSC = 0.9579, Jaccard = 0.9192, Kappa = 0.9579, Hausdorff = 2.2361, Volume T1c = 4743.0, Volume FLAIR = 4855.0
Subject 11: DSC 

In [9]:
# 26.09.2024
# Define the path to the directory containing the subject data
subjects_dir = '/home/linuxlia/Lia_Masterthesis/data/T1mc_vs_T1xFLAIR_controlled'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir)

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs T1xFLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

Subject 05: DSC = 0.9343, Jaccard = 0.8766, Kappa = 0.9342, Hausdorff = 2.2361, Volume T1c = 3752, Volume FLAIR = 3824
Subject 09: DSC = 0.9621, Jaccard = 0.9269, Kappa = 0.9621, Hausdorff = 3.6056, Volume T1c = 3724, Volume FLAIR = 3741
Subject 12: DSC = 0.9206, Jaccard = 0.8530, Kappa = 0.9206, Hausdorff = 4.6904, Volume T1c = 1737, Volume FLAIR = 1741
Subject 04: DSC = 0.8659, Jaccard = 0.7635, Kappa = 0.8659, Hausdorff = 2.2361, Volume T1c = 2576, Volume FLAIR = 2485
Subject 01: DSC = 0.9682, Jaccard = 0.9384, Kappa = 0.9682, Hausdorff = 2.2361, Volume T1c = 4431, Volume FLAIR = 4439
Subject 06: DSC = 0.9319, Jaccard = 0.8726, Kappa = 0.9319, Hausdorff = 3.0000, Volume T1c = 2584, Volume FLAIR = 2500
Subject 03: DSC = 0.9388, Jaccard = 0.8846, Kappa = 0.9388, Hausdorff = 2.8284, Volume T1c = 2127, Volume FLAIR = 2038
Subject 10: DSC = 0.9579, Jaccard = 0.9192, Kappa = 0.9579, Hausdorff = 2.2361, Volume T1c = 4742, Volume FLAIR = 4854
Subject 11: DSC = 0.9444, Jaccard = 0.8947, Kapp

In [4]:
# Define the path to the directory containing the subject data
subjects_dir = '/home/linuxlia/Lia_Masterthesis/data/T1mc_vs_T1xFLAIR_controlled'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir, file_ending1='_ChP_mask_T1_mdc.nii', file_ending2='_ChP_mask_T1.nii')

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs FLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

Subject 05: DSC = 0.4398, Jaccard = 0.2819, Kappa = 0.4397, Hausdorff = 12.2474, Volume T1c = 3752.0, Volume FLAIR = 1146.0
Subject 09: DSC = 0.8360, Jaccard = 0.7182, Kappa = 0.8359, Hausdorff = 6.4031, Volume T1c = 3724.0, Volume FLAIR = 2836.0
Skipping subject pazienti_0_final: Missing data
Subject 12: DSC = 0.6374, Jaccard = 0.4678, Kappa = 0.6374, Hausdorff = 7.8740, Volume T1c = 1737.0, Volume FLAIR = 908.0
Subject 04: DSC = 0.7678, Jaccard = 0.6232, Kappa = 0.7678, Hausdorff = 2.4495, Volume T1c = 2576.9999993999954, Volume FLAIR = 1989.0
Subject 01: DSC = 0.8794, Jaccard = 0.7848, Kappa = 0.8794, Hausdorff = 3.7417, Volume T1c = 4431.999998968095, Volume FLAIR = 3498.0
Subject 06: DSC = 0.3995, Jaccard = 0.2496, Kappa = 0.3995, Hausdorff = 22.2036, Volume T1c = 2584.0, Volume FLAIR = 660.0
Subject 03: DSC = 0.7637, Jaccard = 0.6177, Kappa = 0.7637, Hausdorff = 6.3246, Volume T1c = 2127.0, Volume FLAIR = 1356.0
Subject 10: DSC = 0.8955, Jaccard = 0.8107, Kappa = 0.8954, Hausdorf

In [10]:
# Define the path to the directory containing the subject data
subjects_dir = '/home/linuxlia/Lia_Masterthesis/data/pazienti'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir, file_ending1 = '_ChP_mask_T1_manual_seg.nii', file_ending2 = '_ChP_mask_T1xFLAIR_manual_seg.nii')


Skipping subject 101: Missing data
Skipping subject 079: Missing data
Subject 032: DSC = 0.8301, Jaccard = 0.7096, Kappa = 0.8300, Hausdorff = 9.9499, Volume T1c = 3256.0, Volume FLAIR = 3919.0
Subject 015: DSC = 0.7837, Jaccard = 0.6444, Kappa = 0.7837, Hausdorff = 7.0000, Volume T1c = 1830.0, Volume FLAIR = 1860.0
Subject 002: DSC = 0.8805, Jaccard = 0.7864, Kappa = 0.8804, Hausdorff = 11.0000, Volume T1c = 2484.0, Volume FLAIR = 2309.0
Skipping subject 098: Missing data
Subject 025: DSC = 0.9072, Jaccard = 0.8302, Kappa = 0.9072, Hausdorff = 8.7750, Volume T1c = 3985.0, Volume FLAIR = 3949.0
Subject 043: DSC = 0.8739, Jaccard = 0.7761, Kappa = 0.8739, Hausdorff = 1.7321, Volume T1c = 1774.0, Volume FLAIR = 1526.0
Subject 005: DSC = 0.9071, Jaccard = 0.8300, Kappa = 0.9070, Hausdorff = 7.3485, Volume T1c = 3602.0, Volume FLAIR = 3253.0
Skipping subject 048: Missing data
Skipping subject 049: Missing data
Subject 021: DSC = 0.8385, Jaccard = 0.7219, Kappa = 0.8385, Hausdorff = 5.3852,

In [11]:
# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs FLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")



Wilcoxon signed-rank test results:
Volumes: statistic=311.0, p-value=0.05049099592474704

Paired t-test result:
Volumes T1c vs FLAIR: statistic=1.1235842791416, p-value=0.26757080476773304


In [ ]:
def perform_ttest(path):
    t1cm_volumes = []
    t1xflair_volumes = []

    # List all directories in the given path
    subject_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

    for id in subject_dirs:
        t1cm_file = os.path.join(path, id, f"{id}_t1cm.nii")
        t1xflair_file = os.path.join(path, id, f"{id}_t1xflair.nii")

        if os.path.exists(t1cm_file) and os.path.exists(t1xflair_file):
            t1cm_volumes.append(calculate_volume(t1cm_file))
            t1xflair_volumes.append(calculate_volume(t1xflair_file))

    ttest = stats.ttest_ind(t1cm_volumes, t1xflair_volumes)
    return ttest

In [ ]:
# Function to perform t-test on the volume of T1Cm and T1xFLAIR

def ttest_volume(t1cm_volume, t1xflair_volume):
    
    t1cm_volume = np.array(t1cm_volume)
    t1xflair_volume = np.array(t1xflair_volume)
    ttest = stats.ttest_ind(t1cm_volume, t1xflair_volume)
    return ttest

In [10]:
# Define the path to the directory containing the subject data
subjects_dir = '/home/linuxlia/Lia_Masterthesis/data/T1mc_vs_T1xFLAIR_controlled_OK'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir, file_ending1='_ChP_mask_T1_mdc.nii', file_ending2='_ChP_mask_T1.nii')

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs T1: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

Subject 05: DSC = 0.6553, Jaccard = 0.4873, Kappa = 0.6552, Hausdorff = 7.8740, Volume T1c = 3752, Volume FLAIR = 2346
Subject 09: DSC = 0.8360, Jaccard = 0.7182, Kappa = 0.8359, Hausdorff = 6.4031, Volume T1c = 3724, Volume FLAIR = 2836
Subject 12: DSC = 0.6581, Jaccard = 0.4904, Kappa = 0.6580, Hausdorff = 7.8740, Volume T1c = 1737, Volume FLAIR = 971
Subject 04: DSC = 0.7731, Jaccard = 0.6301, Kappa = 0.7730, Hausdorff = 4.1231, Volume T1c = 2576, Volume FLAIR = 2116
Subject 01: DSC = 0.8870, Jaccard = 0.7970, Kappa = 0.8870, Hausdorff = 3.0000, Volume T1c = 4431, Volume FLAIR = 3701
Subject 06: DSC = 0.6121, Jaccard = 0.4410, Kappa = 0.6120, Hausdorff = 18.0278, Volume T1c = 2584, Volume FLAIR = 1213
Subject 03: DSC = 0.8042, Jaccard = 0.6725, Kappa = 0.8041, Hausdorff = 3.3166, Volume T1c = 2127, Volume FLAIR = 1534
Subject 10: DSC = 0.8955, Jaccard = 0.8107, Kappa = 0.8954, Hausdorff = 3.7417, Volume T1c = 4742, Volume FLAIR = 4050
Subject 11: DSC = 0.7383, Jaccard = 0.5852, Kapp

In [11]:
import pandas as pd
# Create a dictionary with the results
results_t1ce_vs_t1 = {
    'DSC': dsc_scores,
    'Jaccard': jaccard_scores,
    'Kappa': kappa_scores,
    'Hausdorff': hausdorff_scores,
    'Volume_T1c': volumes_t1mc,
    'Volume_T1xFLAIR': volumes_t1xflair
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(results_t1ce_vs_t1)

# Save the DataFrame to a CSV file
df.to_csv('/home/linuxlia/Lia_Masterthesis/phuse_thesis_2024/thesis_experiments/00_preanalysis/results_t1ce_vs_t1.csv', index=False)

In [13]:
# Define the path to the directory containing the subject data
print("T1mc vs T1xFLAIR")
subjects_dir = '/home/linuxlia/Lia_Masterthesis/data/T1mc_vs_T1xFLAIR_controlled_OK'

# Process the subjects and compute statistical measures
dsc_scores, jaccard_scores, kappa_scores, hausdorff_scores, volumes_t1mc, volumes_t1xflair = process_subjects(subjects_dir)

# Perform statistical tests
# Wilcoxon signed-rank test
wilcoxon_volumes = wilcoxon(volumes_t1mc, volumes_t1xflair)

# Paired t-test
ttest_volumes = ttest_rel(volumes_t1mc, volumes_t1xflair)

print("\nWilcoxon signed-rank test results:")
print(f"Volumes: statistic={wilcoxon_volumes.statistic}, p-value={wilcoxon_volumes.pvalue}")

print("\nPaired t-test result:")
print(f"Volumes T1c vs T1xFLAIR: statistic={ttest_volumes.statistic}, p-value={ttest_volumes.pvalue}")

T1mc vs T1xFLAIR
Subject 05: DSC = 0.9176, Jaccard = 0.8477, Kappa = 0.9176, Hausdorff = 2.4495, Volume T1c = 3752, Volume FLAIR = 3687
Subject 09: DSC = 0.9621, Jaccard = 0.9269, Kappa = 0.9621, Hausdorff = 3.6056, Volume T1c = 3724, Volume FLAIR = 3741
Subject 12: DSC = 0.9206, Jaccard = 0.8530, Kappa = 0.9206, Hausdorff = 4.6904, Volume T1c = 1737, Volume FLAIR = 1741
Subject 04: DSC = 0.8659, Jaccard = 0.7635, Kappa = 0.8659, Hausdorff = 2.2361, Volume T1c = 2576, Volume FLAIR = 2485
Subject 01: DSC = 0.9546, Jaccard = 0.9132, Kappa = 0.9546, Hausdorff = 2.2361, Volume T1c = 4431, Volume FLAIR = 4162
Subject 06: DSC = 0.9319, Jaccard = 0.8726, Kappa = 0.9319, Hausdorff = 3.0000, Volume T1c = 2584, Volume FLAIR = 2500
Subject 03: DSC = 0.9388, Jaccard = 0.8846, Kappa = 0.9388, Hausdorff = 2.8284, Volume T1c = 2127, Volume FLAIR = 2038
Subject 10: DSC = 0.9579, Jaccard = 0.9192, Kappa = 0.9579, Hausdorff = 2.2361, Volume T1c = 4742, Volume FLAIR = 4854
Subject 11: DSC = 0.9229, Jacca

In [14]:
# save the results in a csv file
import pandas as pd

# Create a dictionary with the results
results = {
    'DSC': dsc_scores,
    'Jaccard': jaccard_scores,
    'Kappa': kappa_scores,
    'Hausdorff': hausdorff_scores,
    'Volume_T1c': volumes_t1mc,
    'Volume_T1xFLAIR': volumes_t1xflair
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df.to_csv('/home/linuxlia/Lia_Masterthesis/phuse_thesis_2024/thesis_experiments/00_preanalysis/results_t1ce_vs_t1xflair.csv', index=False)